# Simulating sensor degradation filtering

This builds on the solution found in [the notebook exploring different ways to solve the VI problem (via fixing the parametric form of the surrogate distribution)](./exploring_vi_solutions.ipynb).

## Analyzing different $n$, $t$, $f$ combinations
Setting up various combinations of data.

In [1]:

import notebook_modules.general_module as gm
import notebook_modules.vi_simulation_module as vsm
import numpy as np

"""
Define random past values
"""
mu = [
    np.array([0.75, 0.75]),
    np.array([0.55, 0.95]),
    np.array([0.95, 0.55])
]

sigma = [
    # np.array([[1, 0.1], [0.1, 1]]),
    np.array([[2, 0.1], [0.1, 1]]),
    np.array([[1, 0.1], [0.1, 2]])
]

curr_obs = [
    gm.Observation(18, 35), # observed mixed number of tile colors
    gm.Observation(6, 35), # observed mostly white tiles
    gm.Observation(33, 35) # observed mostly black tiles
]
past_f = [
    0.95, # strongly black tile majority
    0.55 # almost no majority
]

param_combo = gm.ParamCombo(mu, sigma, curr_obs, past_f)


Defining models

In [2]:
"""
Define models
"""
# Sensor degradation model and covariance (slow degradation)
A = np.asarray([[1-gm.ZERO_APPROX/1e3, 0],
                [0, 1-gm.ZERO_APPROX/1e3]])
R = np.asarray([[1, 0],
                [0, 1]])

Defining cost functions

In [3]:
from scipy import stats, integrate, optimize

# Define integration limits (i.e., support) for the expectation
limits = [gm.ZERO_APPROX, 1-gm.ZERO_APPROX]

# Cost function for the MAP estimate
def MAP_cost_function(x, joint_dist: gm.MultiVarJointDist):
    return -joint_dist.pdf(x)

# Warm started ELBO
def warm_start_neg_ELBO(C, mu, sigma, joint_dist: gm.MultiVarJointDist):
    """Warm started ELBO using the MAP estimates of the joint distribution
    """

    if C.size == 3:
        cov = gm.reconstruct_cov_mat(C)
        surrogate_dist = stats.multivariate_normal(mean=mu, cov=cov) # params has 3 elements
    elif C.size == 2: surrogate_dist = stats.multivariate_normal(mean=mu, cov=np.diag(C)) # params has 2 elements
    elif C.size == 1:
        sigma[1,0] = C*np.sqrt(sigma[0,0] * sigma[1,1])
        sigma[0,1] = sigma[1,0]
        surrogate_dist = stats.multivariate_normal(mean=mu, cov=sigma) # params has 3 elements
    else: raise

    surrogate_norm_const, _ = integrate.dblquad(
        lambda x, y: surrogate_dist.pdf(np.array([x, y])), *limits, *limits
    )

    ln_expectation, _ = integrate.dblquad(
        lambda x, y: surrogate_dist.pdf(np.array([x, y])) * 
                     (
                         np.log(joint_dist.pdf(np.array([x, y]))) -
                         np.log(surrogate_dist.pdf(np.array([x, y]))) +
                         np.log(surrogate_norm_const)
                     ), *limits, *limits
    )
    return -(ln_expectation)/surrogate_norm_const

Run experiments

In [4]:
sim_data = vsm.SimDataVI(param_combo, A, R)

support_step_size = 1e-3
support = np.arange(gm.ZERO_APPROX, 1-gm.ZERO_APPROX, support_step_size) # n points
x, y = np.meshgrid(support, support)
pos = np.vstack((x.flatten(), y.flatten())).T # (n*n) x 2 vector of coordinate points
xlab = r"$\theta_1$"
ylab = r"$\theta_2$"

# Function to check whether eigenvalues of the covariance matrix are positive
def pos_def_constraint(params):
    if params.size == 3: cov_matrix_part = gm.reconstruct_cov_mat(params) # 3 values
    else: cov_matrix_part = np.diag(params) # 2 values
    return (np.linalg.eigvals(cov_matrix_part) - np.array([1e-6, 1e-6])).ravel()

# Define positive definiteness constraint
pos_def_constraint_dict = {"type": "ineq", "fun": pos_def_constraint}

for i, pc in enumerate(sim_data.get_param_combo().get_params()):

    # joint dist: p(theta_t, n_t, f)
    joint_dist = gm.MultiVarJointDist(dim=2)

    # Unpack params
    past_est = gm.Estimate(*pc[0])
    curr_obs = gm.Observation(*pc[1])
    past_f = pc[2]

    print("past_est.x={0}\npast_est.covar={1}\ncurr_obs.n={2}\ncurr_obs.t={3}\npast_f={4}"
          .format(past_est.x.flatten(),
                  past_est.covar.flatten(),
                  curr_obs.n,
                  curr_obs.t,
                  past_f))

    # Execute prediction step
    predicted_est = gm.prediction_step(past_est, A, R) # index 0 is the past estimate

    # Update joint distribution parameters
    joint_dist.update_binom_params(curr_obs.n, past_f, curr_obs.t)
    joint_dist.update_norm_params(predicted_est.x, predicted_est.covar)
    joint_dist.compute_norm_const()


    """Optimize over mean value"""
    x0_map = predicted_est.x

    map_result = optimize.minimize(
        MAP_cost_function,
        x0_map,
        args=(joint_dist),
        method="SLSQP",
        bounds=[(gm.ZERO_APPROX,1-gm.ZERO_APPROX), (gm.ZERO_APPROX,1-gm.ZERO_APPROX)]
    )

    print("\n###### Optimization results ######\n")
    print(map_result)
    print("Estimated mean:\n", map_result.x)
    print("Cost", -map_result.fun)
    print("\n####################################\n")


    """Optimize over diagonal covariance values"""
    bnds_warm_start_a = [*[(gm.ZERO_APPROX, np.inf) for _ in range(2)]] # 2 variables: c_11, c_22

    # Define initial guess as predicted covariance
    x0_warm_start_a = [predicted_est.covar[0,0], predicted_est.covar[1,1]]

    warm_start_result_a = optimize.minimize(
        warm_start_neg_ELBO,
        x0_warm_start_a,
        args=(map_result.x, None, joint_dist),
        method="SLSQP",
        bounds=bnds_warm_start_a,
        constraints=pos_def_constraint_dict
    )

    print("\n###### Optimization result ######\n")
    print(warm_start_result_a)
    print("Estimated diagonal covariance matrix:\n", np.diag(warm_start_result_a.x))
    print("ELBO", -warm_start_result_a.fun)
    print("\n####################################\n")


    """Optimize over correlation value"""
    # Define positive definiteness constraint for correlation seach
    def pos_def_constraint_corr(corr):
        cov_matrix_part = np.diag(warm_start_result_a.x)
        cov_matrix_part[0,1] = corr[0]
        cov_matrix_part[1,0] = cov_matrix_part[0,1]
        return (np.linalg.eigvals(cov_matrix_part) - np.array([1e-6, 1e-6])).ravel()

    pos_def_constraint_corr_dict = {"type": "ineq", "fun": pos_def_constraint_corr}

    # Define bounds for the correlation search
    bnds_warm_start_b = [(-1, 1)] # 3 variables: c_11, c_22, correlation

    # Define initial guess
    x0_warm_start_b = [0.]

    warm_start_result_b = optimize.minimize(
        warm_start_neg_ELBO,
        x0_warm_start_b,
        args=(map_result.x, np.diag(warm_start_result_a.x), joint_dist),
        method="SLSQP",
        bounds=bnds_warm_start_b,
        constraints=pos_def_constraint_corr_dict,
    )

    print("\n###### Optimization result ######\n")
    print(warm_start_result_b)
    print("Estimated correlation:\n", warm_start_result_b.x)
    print("ELBO", -warm_start_result_b.fun)
    print("\n####################################\n")


    """Compute distribution"""
    # Surrogate distribution; normalized to support
    covar_final = np.diag(warm_start_result_a.x)
    covar_final[0,1] = warm_start_result_b.x*np.sqrt(covar_final[0,0] * covar_final[1,1])
    covar_final[1,0] = covar_final[0,1]
    surr_dist = gm.TruncatedBivariateNormal(mu=map_result.x, sigma=covar_final)
    surr_dist_norm_const = integrate.simpson(
        integrate.simpson(surr_dist.pdf(pos, False).reshape(x.shape), support, axis=1),
        support
    )
    surr_dist.set_norm_const(surr_dist_norm_const)

    # Store data
    sim_data.store_data_pair(
        pc,
        joint_dist,
        surr_dist,
        [map_result, warm_start_result_a, warm_start_result_b]
    )

# Pickle data
sim_data.save()

past_est.x=[0.75 0.75]
past_est.covar=[2.  0.1 0.1 1. ]
curr_obs.n=18
curr_obs.t=35
past_f=0.95

###### Optimization results ######

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -4.763154686533222
       x: [ 5.288e-01  7.504e-01]
     nit: 5
     jac: [ 7.069e-05 -3.207e-05]
    nfev: 19
    njev: 5
Estimated mean:
 [0.52879946 0.75037592]
Cost 4.763154686533222

####################################


###### Optimization result ######

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.02910966045425303
       x: [ 7.167e-03  1.964e+00]
     nit: 11
     jac: [ 3.454e-03  6.486e-04]
    nfev: 40
    njev: 11
Estimated diagonal covariance matrix:
 [[0.0071668  0.        ]
 [0.         1.96398016]]
ELBO -0.02910966045425303

####################################


###### Optimization result ######

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.022151554094979303
       x: [ 1

/home/khaiyichin/research/sensor_degradation_filter/.venv/lib/python3.8/site-packages/scipy/optimize/_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "



###### Optimization result ######

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.8143749228338284
       x: [ 2.287e-02  3.147e-02]
     nit: 44
     jac: [-7.110e-03 -1.337e-03]
    nfev: 172
    njev: 43
Estimated diagonal covariance matrix:
 [[0.02286777 0.        ]
 [0.         0.03147499]]
ELBO -0.8143749228338284

####################################


###### Optimization result ######

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.7962331408955843
       x: [ 2.683e-02]
     nit: 4
     jac: [-6.695e-01]
    nfev: 8
    njev: 4
Estimated correlation:
 [0.02682743]
ELBO -0.7962331408955843

####################################

past_est.x=[0.75 0.75]
past_est.covar=[2.  0.1 0.1 1. ]
curr_obs.n=6
curr_obs.t=35
past_f=0.95

###### Optimization results ######

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -4.863312316706792e-10
       x: [ 7.500e-01  7.500e-01]
    